In [1]:
### Thực hiện bổ sung giá trị D/E cho các doanh nghiệp 
import pandas as pd
import numpy as np

def load_excel(file_path, header=1):
    try:
        df = pd.read_excel(file_path, header=header)
        df.fillna(0, inplace=True)
        return df
    except Exception as e:
        print(f"Không thể tải file {file_path}: {e}")
        raise ValueError(f"Lỗi đọc file: {file_path}")

def calculate_de(company_code):
    df_cdkt = load_excel(f'bctc/Can_doi_ke_toan_{company_code}.xlsx')

    total_debt = df_cdkt[df_cdkt['Quý'] == "C. NỢ PHẢI TRẢ"]["Quý 1 - 2024"].values[0]
    total_all_equity = df_cdkt[df_cdkt['Quý'] == "D.VỐN CHỦ SỞ HỮU"]["Quý 1 - 2024"].values[0]

    if total_all_equity == 0:
        return 0
    else:
        return total_debt / total_all_equity 
    
# Thực hiện đọc file data_company.xlsx và tính toán giá trị D/E cho các doanh nghiệp (company_code tại cột 'Mã CP'), lưu giá trị D/E vào cột 'D/E' và lưu kết quả vào file data_company_de.xlsx
df = load_excel('data_company.xlsx', header=0)
df['D/E'] = df['Mã CP'].apply(calculate_de)
df.to_excel('data_company_de.xlsx', index=False)

Không thể tải file bctc/Can_doi_ke_toan_MHC.xlsx: [Errno 2] No such file or directory: 'bctc/Can_doi_ke_toan_MHC.xlsx'


ValueError: Lỗi đọc file: bctc/Can_doi_ke_toan_MHC.xlsx

In [ ]:
import pandas as pd
import numpy as np

def load_excel(file_path, header=1):
    """Hàm tải file Excel"""
    try:
        df = pd.read_excel(file_path, header=header)
        df.fillna(0, inplace=True)
        return df
    except Exception as e:
        print(f"Không thể tải file {file_path}: {e}")
        return None

def calculate_de(company_code):
    """Tính toán tỷ lệ D/E cho một mã CP"""
    # Thử tải file Excel cho mã công ty
    df_cdkt = load_excel(f'bctc/Can_doi_ke_toan_{company_code}.xlsx')
    
    # Nếu không tải được file, trả về None
    if df_cdkt is None:
        return None

    # Lấy dữ liệu tổng nợ và vốn chủ sở hữu
    try:
        current_assets = df_cdkt[df_cdkt['Quý'] == "A- TÀI SẢN NGẮN HẠN"]["Quý 1 - 2024"].values[0]
        current_liabilities = df_cdkt[df_cdkt['Quý'] == "C. NỢ PHẢI TRẢ"]["Quý 1 - 2024"].values[0]
        quick_assets = df_cdkt[df_cdkt['Quý'] == "A- TÀI SẢN NGẮN HẠN"]["Quý 1 - 2024"].values[0] - df_cdkt[df_cdkt['Quý'] == "IV. Hàng tồn kho"]["Quý 1 - 2024"].values[0]
        cash_assets = df_cdkt[df_cdkt['Quý'] == "I. Tiền và các khoản tương đương tiền"]["Quý 1 - 2024"].values[0]
    except (IndexError, KeyError):
        print(f"Lỗi dữ liệu cho mã CP: {company_code}")
        return None

    # Tính D/E, tránh chia cho 0
    if current_liabilities == 0:
        return 0
    else:
        return cash_assets / current_liabilities 

# Thực hiện đọc file data_company.xlsx và tính toán D/E
df = load_excel('data_company.xlsx', header=0)

# Tính D/E và cập nhật vào cột mới
df['QuickRatio'] = df['Mã CP'].apply(calculate_de)

# Xóa các hàng có giá trị QuickRatio là None (tức là các mã CP không thể tải được file)
df_cleaned = df.dropna(subset=['QuickRatio'])

# Lưu kết quả vào file mới
df_cleaned.to_excel('data_company.xlsx', index=False)

print("Quá trình tính toán hoàn tất. File kết quả đã được lưu vào 'data_company.xlsx'.")


Không thể tải file kqhdkd/Ket_qua_hoat_dong_kinh_doanh_VCB.xlsx: [Errno 2] No such file or directory: 'kqhdkd/Ket_qua_hoat_dong_kinh_doanh_VCB.xlsx'


TypeError: 'NoneType' object is not subscriptable

In [10]:
# Đọc từng cột trong file data_company.xlsx, nếu ô trong cột đó chứa giá trị 0 thì xóa dòng đó
df = pd.read_excel('data_company.xlsx')
df = df[(df != 0).all(1)]
df.to_excel('data_company_test.xlsx', index=False)

In [1]:
import pandas as pd
# Đọc từng cột trong file data_company.xlsx, với mỗi cột, tính tứ phân vị Q1, Q2, Q3, từ đó xác định khoảng giá trị ngoại lệ (outlier) và xóa các dòng chứa giá trị ngoại lệ đó
df = pd.read_excel('data_company.xlsx')
for column in df.columns:
    if column == 'Mã CP' or column == 'Sàn' or column == 'Ngành':
        continue
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
df.to_excel('data_company_test.xlsx', index=False)